In [2]:
import requests
import pandas as pd
import json
from bs4 import BeautifulSoup
import os
from tqdm import tqdm
import multiprocessing as mp
import threading

In [3]:
#create Session and authenticate
s = requests.Session()

#authentication for transkribus API
url = 'https://transkribus.eu/TrpServer/rest/auth/login'

#Yes, I know. You can see my password. Just don't do anything stupid with it please ok. Thanks
auth = {'user': 'hannah.casey@epfl.ch', 'pw':'ZqYI5nDW29VjukNWoUHt'}
r = s.post(url, data=auth)
print(r.status_code)

200


In [4]:
#See collections 
col_url = 'https://transkribus.eu/TrpServer/rest/collections/list'
r = s.get(col_url,data=auth)
print(r.json())

[{'type': 'trpCollection', 'colId': 124423, 'colName': 'Hertziana_Read&Search', 'description': 'created by guenter.hackl@transkribus.eu', 'crowdsourcing': False, 'elearning': False, 'pageId': 30709409, 'url': 'https://files.transkribus.eu/Get?fileType=view&id=DTWJIGLNLUSXZUZAWUJXDEBP', 'thumbUrl': 'https://files.transkribus.eu/Get?fileType=thumb&id=DTWJIGLNLUSXZUZAWUJXDEBP', 'nrOfDocuments': 3846, 'role': 'Editor', 'accountingStatus': 1}, {'type': 'trpCollection', 'colId': 192720, 'colName': 'hannah.casey@epfl.ch Collection', 'description': 'hannah.casey@epfl.ch', 'crowdsourcing': False, 'elearning': False, 'pageId': 53243366, 'url': 'https://files.transkribus.eu/Get?fileType=view&id=DAEQZNPMDNXLWKBWOHOHPWDD', 'thumbUrl': 'https://files.transkribus.eu/Get?fileType=thumb&id=DAEQZNPMDNXLWKBWOHOHPWDD', 'nrOfDocuments': 1, 'role': 'Owner', 'accountingStatus': 1}]


In [5]:
#Get collectionID and name
colId = r.json()[0]['colId']
colName = r.json()[0]['colName']
print(colName, '/', colId)

Hertziana_Read&Search / 124423


In [6]:
#Get list of documents
col_url = 'https://transkribus.eu/TrpServer/rest/collections/'+str(colId)+'/list'
r = s.get(col_url)

In [7]:
docIds = pd.read_csv('DocId.csv')

In [8]:
#test getting the metadata for docid 0

test_id = docIds['DocId'][0]
metadata_url = 'https://transkribus.eu/TrpServer/rest/collections/'+str(colId)+'/'+str(test_id)+'/metadata'
r = s.get(metadata_url)
nr_pages, lang, hierarchy = r.json()['nrOfPages'],r.json()['language'], r.json()['hierarchy']

print(nr_pages, lang, hierarchy)

170 ita_old Rome/Rome_after_1800/Anatomia per uso dei pittori e scultori/BV001535558


In [9]:
#test getting the transcription for docid 0

fulldoc_url = 'https://transkribus.eu/TrpServer/rest/collections/'+str(colId)+'/'+str(test_id)+'/fulldoc'
r = s.get(fulldoc_url)

In [10]:
# for now i'll just turn this one XML page to text, idk if the other documents have different xml layouts
def page_to_text(dict_page):
    soup = BeautifulSoup(dict_page, 'xml')

    # may have multiple transcripts of a single page
    transcript_urls = soup.find('transcripts').findChildren('url')
    transcripts = []
    for transcript_url in transcript_urls:
        transcript = s.get(transcript_url.text)
        soup = BeautifulSoup(transcript.content, 'xml')
        words = soup.find_all('Unicode')
        transcripts.append(' '.join([word.text for word in words]))
    return transcripts

                


In [11]:
def transcribe_book(doc_id):

    if os.path.isfile(f'./data/transcripts/{doc_id}.json'):
        print(f'File {doc_id}.json exists, skipping...')
        return
    print(f'transcribing {doc_id}...')
    

    metadata_url = 'https://transkribus.eu/TrpServer/rest/collections/'+str(colId)+'/'+str(doc_id)+'/metadata'
    r = s.get(metadata_url)
    nr_pages = r.json()['nrOfPages']
    
    try:
        lang = r.json()['language']
    except:
        lang = 'Unknown'
    
    try:
        hierarchy = r.json()['hierarchy']
    except:
        hierarchy = 'Unknown'

    book_dict = {'nr_pages': nr_pages, 'language': lang, 'hierarchy': hierarchy, 'pages': []}
    for i in tqdm(range(1, nr_pages+1)):
        page_url = 'https://transkribus.eu/TrpServer/rest/collections/'+str(colId)+'/'+str(doc_id)+'/'+str(i)
        r = s.get(page_url)
        book_dict['pages'].append(page_to_text(r.content))

    with open(f'./data/transcripts/{doc_id}.json','w') as f:
        json.dump(book_dict, f, indent=4)

    print(f'Finished transcribing {doc_id}!')


In [12]:
## parallelize to the max, but i still think using a DB would be better

# mp.freeze_support()
# segments = mp.cpu_count() * 2

# num_entries = docIds.shape[0]
# segment_size = num_entries // segments

# splits = [docIds[x:x+segment_size] for x in range(0, len(docIds), segment_size)]

# def apply_to_split(split):
#     split.apply(transcribe_book, axis=1)

# threads = []

# for split in splits:
#     t = threading.Thread(target=apply_to_split, args=(split,))
#     threads.append(t)
#     t.start()
    
# for t in threads:
#     t.join()

In [13]:
# sequential version
print(docIds.shape[0])
for doc_id in docIds['DocId']:
        transcribe_book(doc_id)

    if os.path.isfile(f'./data/transcripts/{doc_id}.json'):
        print(f'File {doc_id}.json exists, skipping...')
    else: 
        print(f'transcribing {doc_id}...')
        print(f'Finished transcribing {doc_id}!')

  0%|          | 0/300 [00:00<?, ?it/s]

87
File 825300.json exists, skipping...
File 867194.json exists, skipping...
transcribing 867195...
transcribing 867195...


 52%|█████▏    | 157/300 [00:36<00:51,  2.77it/s]